# Miniproject 1

In [ ]:
!pip install pyperplan
!pip install sympy


## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import numpy as np
from sympy import Symbol, And, Or, satisfiable
from pyperplan.pddl.parser import Parser
from pyperplan import grounding, planner
import numpy as np
import os
import tempfile

BLOCKS_DOMAIN = """(define (domain blocks)
    (:requirements :strips :typing)
    (:types block)
    (:predicates 
        (on ?x - block ?y - block)
        (ontable ?x - block)
        (clear ?x - block)
        (handempty)
        (handfull)
        (holding ?x - block)
    )

    (:action pick-up
        :parameters (?x - block)
        :precondition (and
            (clear ?x) 
            (ontable ?x) 
            (handempty)
        )
        :effect (and
            (not (ontable ?x))
            (not (clear ?x))
            (not (handempty))
            (handfull)
            (holding ?x)
        )
    )

    (:action put-down
        :parameters (?x - block)
        :precondition (and 
            (holding ?x)
            (handfull)
        )
        :effect (and 
            (not (holding ?x))
            (clear ?x)
            (handempty)
            (not (handfull))
            (ontable ?x))
        )

    (:action stack
        :parameters (?x - block ?y - block)
        :precondition (and
            (holding ?x) 
            (clear ?y)
            (handfull)
        )
        :effect (and 
            (not (holding ?x))
            (not (clear ?y))
            (clear ?x)
            (handempty)
            (not (handfull))
            (on ?x ?y)
        )
    )

    (:action unstack
        :parameters (?x - block ?y - block)
        :precondition (and
            (on ?x ?y)
            (clear ?x)
            (handempty)
        )
        :effect (and 
            (holding ?x)
            (clear ?y)
            (not (clear ?x))
            (not (handempty))
            (handfull)
            (not (on ?x ?y))
        )
    )
)
"""

BLOCKS_PROBLEM = """(define (problem blocks)
    (:domain blocks)
    (:objects 
        d - block
        b - block
        a - block
        c - block
    )
    (:init 
        (clear a) 
        (on a b) 
        (on b c)
        (on c d)
        (ontable d) 
        (handempty)
    )
    (:goal (and (on d c) (on c b) (on b a)))
)
"""


class SearchAndRescueSimulator:
    """A simulator for a search and rescue problem.

    In search and rescue, a robot must navigate to, pick up, and
    drop off people that are in need of help.

    States are dictionaries with the following attributes:
      "obstacle_map": A numpy array of 0s and 1s, where a 0
        represents free space and a 1 represents an obstacle.
      "robot": A (row, col) representing the robot's loc.
      "hospital_loc": A (row, col) representing the hospital's loc.
      "carrying": The str name of a person being carried,
        or None, if no person is being carried.
      "people": A dict mapping str people names to (row, col)
        locs. If a person is being carried, they do not
        appear in this dict.

    Actions are strs. The following actions are defined:
      "up" / "down" / "left" / "right" : Moves the robot. The
        robot cannot move into obstacles or off the map.
      "pickup-{person}": If the robot is at the person, and if
        the robot is not already carrying someone, picks.
      "dropoff": If the robot is carrying a person, they are
        dropped off at the robot's current location.

    There is one constant initial state. See `get_initial_state`
    and `render_state`.

    Example usage:
      simulator = SearchAndRescueSimulator()
      state = simulator.get_initial_state()
      simulator.pretty_print_state(state)
      action = "down"
      next_state = simulator.get_next_state(state, action)
    """

    def get_initial_state(self):
        obstacle_map = np.array([
            [0, 0, 0, 0, 0, 0, 0],
            [0, 1, 1, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0],
            [0, 0, 1, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 1, 0],
            [0, 1, 0, 0, 1, 0, 0]
        ], dtype=np.uint8)

        robot = (0, 0)  # top left corner
        hospital = (6, 6)  # bottom right corner
        carrying = None
        people = {
            "p1": (4, 0),
            "p2": (6, 0),
            "p3": (0, 6),
            "p4": (3, 3)
        }

        return dict(
            obstacle_map=obstacle_map,
            robot=robot,
            hospital_loc=hospital,
            carrying=carrying,
            people=people
        )

    def draw_env(self, state_arr, state):
        state_arr[state["obstacle_map"] == 1] = "##"
        state_arr[state["hospital_loc"]] = "Ho"
        state_arr[state["robot"]] = "Ro"
        return state_arr

    def draw_locs(self, state_arr, state):
        for person, loc in state["people"].items():
            if loc == state["hospital_loc"]:
                continue
            elif loc == state["robot"]:
                person = "R" + person[-1]
            elif loc == state["hospital_loc"]:
                continue
            state_arr[loc] = person
        return state_arr

    def pretty_print_state(self, state):
        height, width = state["obstacle_map"].shape
        state_arr = np.full((height, width), "  ", dtype=object)
        state_arr = self.draw_env(state_arr, state)
        state_arr = self.draw_locs(state_arr, state)
        # Add padding
        padded_state_arr = np.full((height + 2, width + 2), "##", dtype=object)
        padded_state_arr[1:-1, 1:-1] = state_arr
        state_arr = padded_state_arr
        carrying_str = f"Carrying: {state['carrying']}"
        for row in state_arr:
            print(''.join(row))
        print(carrying_str)
        print()

    def is_valid_location(self, loc_r, loc_c, state):
        if not (0 <= loc_r < state["obstacle_map"].shape[0] and \
                0 <= loc_c < state["obstacle_map"].shape[1]):
            print("WARNING: attempted to move out of bounds, action has no effect.")
            return False

        if state["obstacle_map"][loc_r, loc_c]:
            print("WARNING: attempted to move into an obstacle, action has no effect.")
            return False

        return True

    def get_legal_actions(self, state):
        legal_actions = ["up", "down", "left", "right", "dropoff"]
        for person in state["people"]:
            legal_actions.append(f"pickup-{person}")
        return legal_actions

    def get_next_state(self, state, action):
        legal_actions = self.get_legal_actions(state)
        if action not in legal_actions:
            raise ValueError(f"Unrecognized action {action}. Actions must be one of: {legal_actions}")

        if action in ["up", "down", "left", "right"]:
            dr, dc = {
                "up": (-1, 0),
                "down": (1, 0),
                "left": (0, -1),
                "right": (0, 1),
            }[action]

            r, c = state["robot"]

            if not self.is_valid_location(r + dr, c + dc, state):
                return state
            new_state = self._copy_state(state)
            new_state["robot"] = (r + dr, c + dc)

            return new_state

        elif action.startswith("pickup"):
            person = action.split("-")[1]

            if state["carrying"] is not None:
                print("WARNING: attempted to pick up a person while already carrying someone, action has no effect.")
                return state

            if person not in state["people"] or (state["people"][person] != state["robot"]):
                print("WARNING: attempted to pick up a person not at the robot location, action has no effect.")
                return state

            new_state = self._copy_state(state)
            del new_state["people"][person]
            new_state["carrying"] = person

            return new_state

        assert action == "dropoff"
        if state["carrying"] is None:
            print("WARNING: attempted to dropoff while not carrying anyone, action has no effect.")
            return state

        person = state["carrying"]
        new_state = self._copy_state(state)
        new_state["carrying"] = None
        new_state["people"][person] = state["robot"]

        return new_state

    def _copy_state(self, state):
        return dict(
            obstacle_map=state["obstacle_map"],  # static
            robot=state["robot"],
            hospital_loc=state["hospital_loc"],
            carrying=state["carrying"],
            people=state["people"].copy()
        )


def execute_sar_plan(plan, simulator=None, verbose=True):
    """Execute a plan for search and rescue.

    Args:
      plan: A list of action strs, see SearchAndRescueSimulator.
      simulator: A SearchAndRescueSimulator.
      verbose: If true, print all the states.

    Returns:
      final_state: A SearchAndRescueSimulator state.
    """
    if simulator is None:
        simulator = SearchAndRescueSimulator()
    state = simulator.get_initial_state()
    if verbose:
        simulator.pretty_print_state(state)
    for action in plan:
        state = simulator.get_next_state(state, action)
        if verbose:
            print("Executed action:", action)
            simulator.pretty_print_state(state)
    return state


def get_num_delivered(state):
    num_delivered = 0
    for loc in state["people"].values():
        if loc == state["hospital_loc"]:
            num_delivered += 1
    return num_delivered


def execute_count_num_delivered(plan, verbose=True):
    """Execute a plan for search and rescue and count the number of
      people delivered.

    Args:
      plan: A list of action strs, see SearchAndRescueSimulator.
      verbose: If true, print all the states.

    Returns:
      num_delivered: int
    """
    state = execute_sar_plan(plan, verbose=verbose)
    return get_num_delivered(state)


def check_sar_plan(plan, num_people, verbose=True):
    """Execute a plan for search and rescue and check the goal.

    Args:
      plan: A list of action strs, see SearchAndRescueSimulator.
      num_people: Number of targets to be rescued.
      verbose: If true, print all the states.

    Returns:
      succeeded: bool
    """
    return execute_count_num_delivered(plan, verbose=verbose) == num_people


def executeobs_count_num_delivered(plan, verbose=True):
    """Execute a plan for search and rescue and count the number of
      people delivered.

    Args:
      plan: A list of action strs, see Obs_SearchAndRescueSimulator.
      verbose: If true, print all the states.

    Returns:
      num_delivered: int
    """
    simulator = Obs_SearchAndRescueSimulator()
    state = execute_sar_plan(plan, simulator, verbose=verbose)
    return get_num_delivered(state)


def check_obs_sar_plan(plan, num_people, verbose=True):
    """Execute a plan for search and rescue and check the goal.

    Args:
      plan: A list of action strs, see SearchAndRescueSimulator.
      num_people: Number of targets to be rescued.
      verbose: If true, print all the states.

    Returns:
      succeeded: bool
    """
    return executeobs_count_num_delivered(plan, verbose=verbose) == num_people


def run_planning(domain_pddl_str, problem_pddl_str, search_alg_name,
                 heuristic=None):
    """Plan a sequence of actions to solve the given PDDL problem.

    This function is a lightweight wrapper around pyperplan.

    Args:
      domain_pddl_str: A str, the contents of a domain.pddl file.
      problem_pddl_str: A str, the contents of a problem.pddl file.
      search_alg_name: A str, the name of a search algorithm in
        pyperplan. Options: astar, wastar, gbf, bfs, ehs, ids, sat.
      heuristic: A str or a pyperplan `Heuristic` class.
        A str, the name of a heuristic in pyperplan.
          Options: blind, hadd, hmax, hsa, hff, lmcut, landmark.
        A pyperplan `Heuristic` class.
          See: https://github.com/aibasel/pyperplan/blob/main/doc/documentation.md#implementing-new-heuristics

    Returns:
      plan: A list of actions; each action is a pyperplan Operator.
    """
    # Parsing the PDDL
    domain_file = tempfile.NamedTemporaryFile(delete=False)
    problem_file = tempfile.NamedTemporaryFile(delete=False)
    with open(domain_file.name, 'w') as f:
        f.write(domain_pddl_str)
    with open(problem_file.name, 'w') as f:
        f.write(problem_pddl_str)
    parser = Parser(domain_file.name, problem_file.name)
    domain = parser.parse_domain()
    problem = parser.parse_problem(domain)
    os.remove(domain_file.name)
    os.remove(problem_file.name)

    # Ground the PDDL
    task = grounding.ground(problem)

    # Get the search alg
    search_alg = planner.SEARCHES[search_alg_name]

    if heuristic is None:
        return search_alg(task)

    if isinstance(heuristic, str):
        # Get the heuristic from pyperplan
        heuristic_initialized = planner.HEURISTICS[heuristic](task)
    else:
        # Else, use customized heuristic
        heuristic_initialized = heuristic(task)

    # Run planning
    return search_alg(task, heuristic_initialized)





## Let's Make a Plan


### Question
Use `run_planning` to find a plan for the blocks problem defined at the top of the colab file (`BLOCKS_DOMAIN`, `BLOCKS_PROBLEM`).

  The `run_planning` function takes in a PDDL domain string, a PDDL problem string, the name of a search algorithm, and the name of a heuristic (if the search algorithm is informed) or a customized heuristic class. It then uses the Python planning library `pyperplan` to find a plan.

  The plan returned by `run_planning` is a list of pyperplan Operators. You should not need to manipulate these data structures directly in this homework, but if you are curious about the definition, see [here](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py#L23).

  The search algs available in pyperplan are: `astar, wastar, gbf, bfs, ehs, ids, sat`. The heuristics available in pyperplan are: `blind, hadd, hmax, hsa, hff, lmcut, landmark`.

  For this question, use the `astar` search algorithm with the `lmcut` heuristic.

For reference, our solution is **1** line(s) of code.

In [ ]:

def planning_warmup():
  """Use run_planning to find a plan for the blocks problem
    defined at the top of the colab file (BLOCKS_DOMAIN, BLOCKS_PROBLEM).

    Use the astar search algorithm with the lmcut heuristic.

  Returns:
    plan: A list of actions; each action is a pyperplan Operator.    
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def warmup_test1():
    plan = planning_warmup()
    assert len(plan) == 8
    assert plan[0].name == '(unstack a b)'

warmup_test1()

print('Tests passed.')

## Fill in the Blanks


### Question
You've received PDDL domain and problem strings from your boss and you need to make a plan, pronto! Unfortunately, some of the PDDL is missing.

  Here's what you know. What you're trying to model is a newspaper delivery robot. The robot starts out at a "home base" where there are papers that it can pick up. The robot can hold arbitrarily many papers at once. It can then move around to different locations and deliver papers.

  Not all locations want a paper -- the goal is to satisfy all the locations that do want a paper.

  You also know:
  * There are 6 locations in addition to 1 for the homebase. Locations 1, 2, 3, and 4 want paper; locations 5 and 6 do not.
  * There are 8 papers at the homebase.
  * The robot is initially at the homebase with no papers packed.

  Use this description to complete the PDDL domain and problem.

  If you are running into issues debugging the PDDL, look ahead to the next question, where we describe a useful online PDDL editor, and some common PDDL writing pitfalls.
  

For reference, our solution is **88** line(s) of code.

In [ ]:

def pddl_warmup():
    """Creates a PDDL domain and problem strs for newspaper delivery.

    Returns:
      domain: str
      problem: str
    """
    domain_str = """(define (domain newspapers)
    (:requirements :strips :typing)
    (:types loc paper)
    (:predicates 
      (isHomeBase ?loc - loc)
      ; TODO: Add missing predicates!
    )

    (:action pick-up
      :parameters ()    ; TODO: Add missing parameters!
      :precondition (and
        (at ?loc)
        (isHomeBase ?loc)
        (unpacked ?paper)
      )
      :effect (and
        (not (unpacked ?paper))
        (carrying ?paper)
      )
    )

    (:action move
      :parameters (?from - loc ?to - loc)
      :precondition (and
        (at ?from) 
      )
      :effect (and
        (not (at ?from))
        (at ?to)
      )
    )

    (:action deliver
      :parameters (?paper - paper ?loc - loc)
      :precondition (and
        ; TODO: Add missing preconditions!
      )
      :effect (and
        ; TODO: Add missing effects!
      )
    )

)"""

    problem_str = """(define (problem newspapers1) (:domain newspapers)
  (:objects
    loc-0 - loc
    loc-1 - loc
    loc-2 - loc
    loc-3 - loc
    loc-4 - loc
    loc-5 - loc
    loc-6 - loc
    paper-0 - paper
    paper-1 - paper
    paper-2 - paper
    paper-3 - paper
    paper-4 - paper
    paper-5 - paper
    paper-6 - paper
    paper-7 - paper
  )
  (:init 
    (at loc-0)
    (unpacked paper-0)
    ; TODO: Add missing initial atoms!
  )
  (:goal (and
    (satisfied loc-1)
    (satisfied loc-2)
    (satisfied loc-3)
    (satisfied loc-4)
  ))
)"""

    return domain_str, problem_str

### Tests

In [ ]:
def warmup_test2():
    domain, problem = pddl_warmup()
    plan = run_planning(domain, problem, "gbf", "hadd")
    assert plan, "Failed to find a plan."
    picked_up_papers = set()
    satisfied_locs = set()
    for op in plan:
        if "pickup" in op.name:
            _, _, paper, _ = op.name.split(" ")
            assert paper not in picked_up_papers, \
                "Should not pick up the same paper twice"
            picked_up_papers.add(paper)
        elif "deliver" in op.name:
            _, loc = op.name.rsplit(" ", 1)
            assert loc.endswith(")")
            loc = loc[:-1]
            assert loc not in satisfied_locs, \
                "Should not deliver to the same place twice"
            satisfied_locs.add(loc)
    assert satisfied_locs == {"loc-1", "loc-2", "loc-3", "loc-4"}

warmup_test2()

print('Tests passed.')

## Search and Rescue Warmup 1


### Question
Find the initial robot location in a SearchAndRescueSimulator.

For reference, our solution is **2** line(s) of code.

In [ ]:

def sar_warmup1(simulator):
  """Find the initial robot location in the SearchAndRescueSimulator.

  Args:
    simulator: A SearchAndRescueSimulator.

  Returns:
    robot_loc: A tuple of ints (row, col) representing the robot state.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sar_warmup_test1():
    simulator = SearchAndRescueSimulator()
    robot_loc = sar_warmup1(simulator)
    assert robot_loc == (0, 0)

sar_warmup_test1()

print('Tests passed.')

## Search and Rescue Warmup 2


### Question
Check if a row and col have an obstacle in a SearchAndRescueSimulator state.

For reference, our solution is **1** line(s) of code.

In [ ]:

def sar_warmup2(sar_state, row, col):
  """Check if a row and col have an obstacle in a SearchAndRescueSimulator state.

  Args:
    sar_state: A SearchAndRescueSimulator state.
    row: An int.
    col: An int.

  Returns:
    has_obstacle: True if (row, col) has an obstacle in sar_state.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sar_warmup_test2():
    simulator = SearchAndRescueSimulator()
    state = simulator.get_initial_state()
    assert sar_warmup2(state, 0, 0) == False
    assert sar_warmup2(state, 0, 1) == False
    assert sar_warmup2(state, 1, 1) == True
    assert sar_warmup2(state, 1, 2) == True

sar_warmup_test2()

print('Tests passed.')

## Search and Rescue Warmup 3


### Question
Hand-code a list of actions that will deliver person 'p1' to the hospital location.

For reference, our solution is **1** line(s) of code.

In [ ]:

def sar_warmup3():
  """Hand-code a list of actions that will deliver person 'p1' to the hospital location.

  Returns:
    actions: A list of str actions that will take person p1 to the hospital loccation.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sar_warmup_test3():
    assert execute_count_num_delivered(sar_warmup3()) == 1

sar_warmup_test3()

print('Tests passed.')

## Search and Rescue


### Question
Make a plan to solve the search and rescue problem in SearchAndRescueSimulator.

We need to implement a function to find such a plan. When the output of this function is given to `check_sar_plan`, it should return True.

This function should do the following:

1. Create PDDL domain and problem strings for search and rescue.  The operators should work for any grid size, obstacles, people locations, and hospital location.
2. Invoke `run_planning` with the `gbf` search algorithm with the `lmcut` heuristic.
3. Convert the output of run_planning (pyperplan Operators) into actions that can be given to the SearchAndRescueSimulator.

For reference, this function takes ~1-2 seconds to run with our implementation.
To get credit on catsoop, make sure that your function finishes in <10 seconds.

If you're not sure how on how to get started, scroll down to the bottom of the catsoop page to see a hint.

**Notes**:
* In this problem, you will need to constructs somewhat complicated strings.  We *strongly* encourage you to read about [Python-3 f-strings](https://www.digitalocean.com/community/tutorials/how-to-use-f-strings-to-create-strings-in-python-3) which make this process much easier than the alternatives.
* You may find `simulator.pretty_print_state` useful for debugging.
* We also highly recommend printing out the domain and problem after they have been created, and copying them into [editor.planning.domains](http://editor.planning.domains) to check whether it's possible to find a plan.
* The image in catsoop with the robot and the bears is a faithful depiction of the initial state. For example, the initial locations of the people are: "p1": (4, 0), "p2": (6, 0), "p3": (0, 6), "p4": (3, 3).
* One part of this problem that may be initially counterintuitive is the way that we'll represent locations in PDDL.
In the simulator, a location is a tuple of integers. PDDL does not support such representations -- everything needs to be just an object with a string name.
So to represent a location like (3, 5), we will make a string "l3-5" (where the first character there is a lowercase L), and we'll create an object
with that name, of type "location". We will also need a way to encode the fact that the robot can only move between adjacent locations in the grid.
In the simulator, we can compare the numeric values of locations like (3, 5) and (3, 6) to see if they are neighbors.
But in PDDL, all we have are the objects with string names, and we need to encode everything in terms of predicates.
So, we will create a predicate `(conn ?v0 - location ?v1 - location ?v2 - direction)`, which says that location `?v0` is connected to locaction `?v1`
in direction `?v2`. For example, `(conn l3-5 l3-6 right)` might appear in the initial state. We can then use these `conn` predicates in
the preconditions of a `move` operator to encode the fact that the robot can only move between adjacent locations.
* We do not recommend modelling the hospital explicitly with special objects / types / predicates. Instead, the goal should be to deliver all people to the hospital_loc, that is, `l6-6`.
In words, the goal should be "person1 is at l6-6 and person2 is at l6-6 and person3 is at l6-6 and person4 is at l6-6."
  

For reference, our solution is **121** line(s) of code.

In [ ]:

def find_search_and_rescue_plan():
    """Make a plan to solve the search and rescue problem in SearchAndRescueSimulator.

    When the output of this function is given to `check_sar_plan`, it should return True.

    This function should do the following:
      1. Create PDDL domain and problem strings for search and rescue.
      2. Invoke `run_planning` using the `gbf` search algorithm with the `lmcut` heuristic.
      3. Convert the output of run_planning (pyperplan Operators) into actions
        that can be given to the SearchAndRescueSimulator.

    For reference, this function takes ~1-2 seconds to run with our implementation.

    Returns:
      plan: A list of actions; each action is a str, see SearchAndRescueSimulator.
    """
    simulator = SearchAndRescueSimulator()
    init_sim_state = simulator.get_initial_state()

    # <<< TODO: fill in missing parts in the PDDL domain below >>>
    SAR_DOMAIN = """(define (domain searchandrescue)
  (:requirements :typing)
  (:types person location direction)

  (:constants
    down - direction
    left - direction
    right - direction
    up - direction
  )

  (:predicates
    (conn ?v0 - location ?v1 - location ?v2 - direction)
    ; TODO: write more here
  )

  (:action move-robot
    :parameters (?from - location ?to - location ?dir - direction)
    :precondition (and
      (conn ?from ?to ?dir)
      ; TODO: write more here
    )
    :effect (and
      ; TODO: write more here
    )
  )

  (:action pickup-person
    :parameters (?person - person ?loc - location)
    :precondition (and
      ; TODO: write more here
    )
    :effect (and
      ; TODO: write more here
    )
  )

  (:action dropoff-person
    :parameters (?person - person ?loc - location)
    :precondition (and
      ; TODO: write more here
    )
    :effect (and
      ; TODO: write more here
    )
  )
)"""

    # Create objects str
    objects_strs = []
    for r, c in np.argwhere(init_sim_state["obstacle_map"] == 0):
        # Creates one object for all locations in the grid that are
        # not occupied by an obstacle. For example, if r = 0, c=0, then
        # this would create an object l0-0 of type "location". Note that
        # the first character here is a lowercase L.
        objects_strs.append(f"l{r}-{c} - location")
    # <<< TODO: add object strs for people >>>
    objects_str = " ".join(objects_strs)

    # Create init str
    deltas = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1),
    }
    init_strs = []
    # Here we're going to add one (conn ...) atom for every pair
    # of clear adjacent locations. We do not have objects for
    # locations that have obstacles.
    height, width = init_sim_state["obstacle_map"].shape
    for r, c in np.argwhere(init_sim_state["obstacle_map"] == 0):
        for direction, (dr, dc) in deltas.items():
            if not (0 <= r + dr < height and 0 <= c + dc < width):
                continue
            if init_sim_state["obstacle_map"][r + dr, c + dc] == 1:
                continue
            # For example, if r == 0, c == 0, dr == 0, dc == 1, then
            # this line adds the atom (conn l0-0 l0-1 right).
            init_strs.append(f"(conn l{r}-{c} l{r + dr}-{c + dc} {direction})")
    # <<< TODO: add more init strs >>>
    init_str = " ".join(init_strs)

    # Create goal str
    goal_strs = []
    hospital_r, hospital_c = init_sim_state["hospital_loc"]
    # <<< TODO: add goal strs >>>
    goal_str = " ".join(goal_strs)

    SAR_PROBLEM = f"""(define (problem searchandrescue) (:domain searchandrescue)
  (:objects
  {objects_str}
  )
  (:init 
  {init_str}
  )
  (:goal (and {goal_str}))
)"""

    import time
    start_time = time.time()
    plan = run_planning(SAR_DOMAIN, SAR_PROBLEM, "gbf", "lmcut")
    assert plan, "Failed to find a plan."
    print(f"Planning duration: {time.time() - start_time} seconds.")

    # Convert operators to actions
    actions = []
    for op in plan:
        if "move-robot" in op.name:
            _, direction = op.name[:-1].rsplit(" ", 1)
            action = direction
        elif "pickup-person" in op.name:
            _, person, _ = op.name.split(" ")
            action = f"pickup-{person}"
        else:
            assert "dropoff-person" in op.name
            action = "dropoff"
        actions.append(action)

    return actions

### Tests

In [ ]:
def sar_test1():
    assert check_sar_plan(find_search_and_rescue_plan(), 4)

sar_test1()

print('Tests passed.')

## Search algorithm & heuristic comparisons


### Question
Let's now compare different search algorithms and
heuristics on the search and rescue problem above.

The search algs available in pyperplan are: `astar, wastar, gbf, bfs, ehs, ids, sat`.

The heuristics available in pyperplan are: `blind, hadd, hmax, hsa, hff, lmcut, landmark`.

Unfortunately the documentation for pyperplan is limited at the moment, but if you
are curious to learn more about its internals, the code is open-sourced here:

  https://github.com/aibasel/pyperplan

Choose 8 combinations of (search algorithm, heuristic). For each, record the planning
duration in seconds for Search and Rescue. If planning takes more than 30 seconds,
you can kill the process and record "timeout".  Also record the length of the plans.

<div class="question question-multiplechoice">
<b>Submission Material:</b> In your submitted pdf, please include a table with headers
"Search Algorithm", "Heuristic", "Duration (s)", and "Plan Length" with 8 rows.
</div>


## Logic_warmup 1


### Question
Use sympy to determine whether the following formula is satisfiable:
$(\neg x_1 \land x_2) \Rightarrow ((x_2 \lor x_3) \land (x_1 \lor \neg x_3))$.


For reference, our solution is **3** line(s) of code.

In [ ]:

def formula1_is_satisfiable():
  """Determines whether the above formula is satisfiable.

  Returns:
    is_satisfiable: A bool indicating whether the formula is satisfiable.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert formula1_is_satisfiable() == True
print('Tests passed.')

## Logic_warmup 2


### Question
Use sympy to determine whether the following formula is satisfiable:
$(x_1 \lor x_2) \land (\neg x_1 \lor \neg x_2) \land (x_1 \lor \neg x_2) \land (\neg x_1 \lor x_2)$.


For reference, our solution is **3** line(s) of code.

In [ ]:

def formula2_is_satisfiable():
  """Determines whether the above formula is satisfiable.

  Returns:
    is_satisfiable: A bool indicating whether the formula is satisfiable.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert formula2_is_satisfiable() == False
print('Tests passed.')

## Logic_warmup 3


### Question
Prove to yourself that the SAT solver is doing something more clever than enumerating truth tables. Write a formula involving 100 variables for which the SAT solver is able to quickly find a solution. (Hint: assume that the SAT solver is like DPLL. What kinds of formulas would be especially easy for DPLL to satisfy?)

For reference, our solution is **1** line(s) of code.

In [ ]:

def create_large_solvable_formula():
  """Return a sympy formula with at least 100 variables that `satisfiable` can
  quickly solve.

  Returns:
    formula: A sympy logical formula.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def test_large_solvable_formula():
    large_formula = create_large_solvable_formula()
    assert len(large_formula.free_symbols) >= 100
    return bool(satisfiable(large_formula))

test_large_solvable_formula()

print('Tests passed.')

## Search and Rescue Inference


### Question
Write a program that takes a grid as input and infers unknown values.

Your program should output a new grid with all determinable unknown values replaced with the inferred value. If an unknown value cannot be determined, it should be left unknown.

**Your program should use sympy.**


For reference, our solution is **58** line(s) of code.

In [ ]:

def infer_unknown_values(grid):
  """Fill in any unknown values in the grid that can be inferred.

  Args: grid: A list of lists of "F", "U", "S", "W", or "C".
  Returns:
    inferred_grid: A copy of grid with some unknown values replaced.

  Example:
    >> grid = [
    >>   ["F", "U", "C"],
    >>   ["W", "C", "U"],
    >>   ["U", "U", "C"]
    >> ]
    >> infer_unknown_values(grid)
    >> [["F" "S" "C"]
    >>  ["W" "C" "C"]
    >>  ["U" "U" "C"]]
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert infer_unknown_values([["U", "F"]]) == [["S", "F"]]

assert infer_unknown_values([["F", "U", "C"], ["S", "C", "U"], ["U", "U", "C"]]) == [["F", "S", "C"], ["S", "C", "C"], ["U", "U", "C"]]

assert infer_unknown_values([["U", "C", "C"], ["S", "C", "U"], ["U", "U", "C"]]) == [["C", "C", "C"], ["S", "C", "C"], ["F", "S", "C"]]

assert infer_unknown_values([["U", "S", "C", "U"], ["U", "U", "C", "U"], ["U", "S", "C", "U"]]) == [["F", "S", "C", "C"], ["S", "C", "C", "C"], ["F", "S", "C", "C"]]

assert infer_unknown_values([["U", "U", "C", "U", "U", "U", "U", "U"], ["C", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "C", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "F", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"]]) == [["C", "C", "C", "U", "U", "U", "U", "U"], ["C", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "C", "U", "S", "U", "U", "U", "U"], ["U", "U", "S", "F", "S", "U", "U", "U"], ["U", "U", "U", "S", "U", "U", "U", "U"]]

assert infer_unknown_values([["C", "U", "C", "U", "U", "C", "U"], ["U", "W", "W", "U", "C", "W", "W"], ["U", "F", "U", "U", "U", "F", "U"], ["C", "S", "W", "C", "U", "U", "U"], ["U", "U", "W", "U", "W", "U", "U"], ["C", "C", "U", "C", "U", "W", "U"], ["U", "W", "C", "U", "W", "U", "C"]]) == [["C", "C", "C", "C", "C", "C", "C"], ["C", "W", "W", "C", "C", "W", "W"], ["S", "F", "S", "C", "S", "F", "S"], ["C", "S", "W", "C", "C", "S", "U"], ["C", "C", "W", "C", "W", "U", "U"], ["C", "C", "C", "C", "C", "W", "U"], ["C", "W", "C", "C", "W", "C", "C"]]
print('Tests passed.')

## Search and Rescue with Observation


### Question
Formulate the combined PDDL domain for search-and-rescue

For reference, our solution is **229** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `infer_unknown_values`. You may not need to use all of them.

In [ ]:

def find_obs_search_and_rescue_plan(state=None, simulator=None, search_algo='astar', heuristic='lmcut',
                                    return_time=False):
  """Make a plan to solve the search and rescue problem with observation in Obs_SearchAndRescueSimulator.

  When the output of this function is given to `check_obs_sar_plan`, it should return True.

  This function should do the following:
    1. Create PDDL domain and problem strings for search and rescue.
    2. Invoke `run_planning` using the `gbf` search algorithm with the `hadd` heuristic.
    3. Convert the output of run_planning (pyperplan Operators) into actions
      that can be given to the Obs_SearchAndRescueSimulator.

  For reference, this function takes ~1-2 seconds to run with our implementation.

  Returns:
    plan: A list of actions; each action is a str, see Obs_SearchAndRescueSimulator.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def obs_sar_test():
    assert check_obs_sar_plan(find_obs_search_and_rescue_plan(search_algo='gbf',heuristic='lmcut'), 4)

obs_sar_test()

print('Tests passed.')

## Search and Rescue with Replanning


### Question
Formulate the combined PDDL domain for search-and-rescue

For reference, our solution is **360** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `infer_unknown_values`. You may not need to use all of them.

In [ ]:

def find_pobs_search_and_rescue_plan(state, heuristic, return_time=False):
  """Make a plan to solve the search and rescue problem with observation in PObs_SearchAndRescueSimulator.

  When the output of this function is given to `replan_posar_plan`, it should return True.

  This function should do the following:
    1. Create PDDL domain and problem strings for search and rescue.
    2. Invoke `run_planning` using the `astar` search algorithm with the `lmcut` heuristic or customized heuristic.
    3. Convert the output of run_planning (pyperplan Operators) into actions
      that can be given to the PObs_SearchAndRescueSimulator.

  For reference, this function takes ~1-2 seconds to run with our implementation.

  Returns:
    plan: A list of actions; each action is a str, see SearchAndRescueSimulator.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def pobs_sar_test1():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['U', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['U', 'U', 'U', 'C', 'U', 'U', 'U'],
        ['C', 'U', 'W', 'C', 'U', 'U', 'U'],
        ['U', 'U', 'W', 'U', 'W', 'U', 'C'],
        ['C', 'C', 'U', 'C', 'U', 'W', 'U'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['S', 'F', 'S', 'C', 'C', 'C', 'C'],
        ['C', 'S', 'W', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'W', 'C', 'C'],
        ['C', 'C', 'C', 'C', 'C', 'W', 'C'],
        ['C', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (6, 0),
    }
    simulator = PObs_SearchAndRescueSimulator(observation_map=observation_map,
                                              gt_hazard_map=gt_hazard_map, people=people)
    planner = find_pobs_search_and_rescue_plan
    assert replan_posar_plan(simulator, 1, planner, return_time=True)

pobs_sar_test1()

def pobs_sar_test2():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['C', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['C', 'U', 'U', 'C', 'U', 'U', 'U'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['U', 'C', 'W', 'C', 'W', 'U', 'C'],
        ['U', 'U', 'C', 'C', 'U', 'W', 'C'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'W', 'C', 'C'],
        ['C', 'C', 'C', 'C', 'C', 'W', 'C'],
        ['C', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (5, 2),
    }
    simulator = PObs_SearchAndRescueSimulator(observation_map=observation_map,
                                              gt_hazard_map=gt_hazard_map, people=people)
    planner = find_pobs_search_and_rescue_plan
    assert replan_posar_plan(simulator, 1, planner, return_time=True)

pobs_sar_test2()

print('Tests passed.')

## Search and Rescue with Replanning, varying 'look' cost


### Question
Formulate the combined PDDL domain for search-and-rescue

For reference, our solution is **201** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `find_pobs_search_and_rescue_plan`, `infer_unknown_values`. You may not need to use all of them.

In [ ]:

def find_pobs_search_and_rescue_plan_multioperator(state, heuristic, return_time=False):
    # Based on `find_pobs_search_and_rescue_plan`, add pseudo operators to increase the cost of 'look'.
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def pobs_sar_test3():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['C', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['C', 'U', 'U', 'C', 'U', 'U', 'U'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['U', 'C', 'W', 'C', 'W', 'U', 'C'],
        ['U', 'U', 'C', 'C', 'U', 'W', 'C'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'W', 'C', 'C'],
        ['C', 'C', 'C', 'C', 'C', 'W', 'C'],
        ['C', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (5, 2),
    }
    simulator = PObs_SearchAndRescueSimulator(observation_map=observation_map,
                                              gt_hazard_map=gt_hazard_map, people=people)
    assert replan_posar_plan(simulator, 1, planner=find_pobs_search_and_rescue_plan_multioperator,
                             heuristic='lmcut', return_time=True)

pobs_sar_test3()

print('Tests passed.')

## Search and Rescue with Replanning, varying 'look' cost


### Question
Formulate the combined PDDL domain for search-and-rescue

For reference, our solution is **12** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `find_pobs_search_and_rescue_plan`, `infer_unknown_values`. You may not need to use all of them.

In [ ]:

def create_heuristic(look_cost):
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def pobs_sar_test4():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['C', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['C', 'U', 'U', 'C', 'U', 'U', 'U'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['U', 'C', 'W', 'C', 'W', 'U', 'C'],
        ['U', 'U', 'C', 'C', 'U', 'W', 'C'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'W', 'C', 'C'],
        ['C', 'C', 'C', 'C', 'C', 'W', 'C'],
        ['C', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (5, 2),
    }
    simulator = PObs_SearchAndRescueSimulator(observation_map=observation_map,
                                              gt_hazard_map=gt_hazard_map, people=people)
    look_cost2_heuristic = create_heuristic(2)
    assert replan_posar_plan(simulator, 1, planner=find_pobs_search_and_rescue_plan,
                             heuristic=look_cost2_heuristic, return_time=True)

pobs_sar_test4()

def pobs_sar_test5():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['C', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'U', 'C', 'C', 'U'],
        ['C', 'C', 'W', 'U', 'U', 'C', 'C'],
        ['U', 'C', 'W', 'U', 'W', 'U', 'C'],
        ['U', 'U', 'C', 'C', 'U', 'W', 'C'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'W', 'C', 'C'],
        ['C', 'C', 'C', 'C', 'C', 'W', 'C'],
        ['C', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (2, 2),
    }
    simulator = PObs_SearchAndRescueSimulator(observation_map=observation_map,
                                              gt_hazard_map=gt_hazard_map, people=people)
    look_cost_heuristic = create_heuristic(8)
    assert replan_posar_plan(simulator, 1, planner=find_pobs_search_and_rescue_plan,
                             heuristic=look_cost_heuristic, return_time=True)

pobs_sar_test5()

def pobs_sar_test6():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['C', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'U', 'C', 'C', 'U'],
        ['C', 'C', 'W', 'U', 'U', 'C', 'C'],
        ['U', 'C', 'W', 'U', 'W', 'U', 'C'],
        ['U', 'U', 'C', 'C', 'U', 'W', 'C'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'W', 'C', 'C'],
        ['C', 'C', 'C', 'C', 'C', 'W', 'C'],
        ['C', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (2, 2),
    }
    simulator = PObs_SearchAndRescueSimulator(observation_map=observation_map,
                                              gt_hazard_map=gt_hazard_map, people=people)
    look_cost_heuristic = create_heuristic(5)
    assert replan_posar_plan(simulator, 1, planner=find_pobs_search_and_rescue_plan,
                             heuristic=look_cost_heuristic, return_time=True)

pobs_sar_test6()

print('Tests passed.')

## Search and Rescue with Replanning, with operators to look at neighbouring squares


### Question
Formulate the combined PDDL domain for search-and-rescue

For reference, our solution is **368** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `find_pobs_search_and_rescue_plan`, `infer_unknown_values`. You may not need to use all of them.

In [ ]:

def find_pobs_search_and_rescue_plan(state, heuristic, return_time=False):
  """Make a plan to solve the search and rescue problem with observation in PObs_SearchAndRescueSimulator.

  When the output of this function is given to `replan_posar_plan`, it should return True.

  This function should do the following:
    1. Create PDDL domain and problem strings for search and rescue.
    2. Invoke `run_planning` using the `astar` search algorithm with the `lmcut` heuristic or customized heuristic.
    3. Convert the output of run_planning (pyperplan Operators) into actions
      that can be given to the PObs_SearchAndRescueSimulator.

  For reference, this function takes ~1-2 seconds to run with our implementation.

  Returns:
    plan: A list of actions; each action is a str, see SearchAndRescueSimulator.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def pobs_sar_test6():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['C', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'U', 'C', 'C', 'U'],
        ['C', 'C', 'W', 'U', 'U', 'C', 'C'],
        ['U', 'C', 'W', 'U', 'W', 'U', 'C'],
        ['U', 'U', 'C', 'C', 'U', 'W', 'C'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'W', 'C', 'C'],
        ['C', 'C', 'C', 'C', 'C', 'W', 'C'],
        ['C', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (2, 2),
    }
    simulator = PObs_SearchAndRescueSimulator(observation_map=observation_map,
                                              gt_hazard_map=gt_hazard_map, people=people)
    look_cost_heuristic = create_heuristic(5)
    assert replan_posar_plan(simulator, 1, planner=find_pobs_search_and_rescue_plan,
                             heuristic=look_cost_heuristic, return_time=True)

pobs_sar_test6()

print('Tests passed.')

## Search and Rescue with Replanning, with operators to look at neighbouring squares


### Question
Formulate the combined PDDL domain for search-and-rescue

For reference, our solution is **90** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `find_pobs_search_and_rescue_plan_multioperator`, `find_pobs_search_and_rescue_plan`, `infer_unknown_values`. You may not need to use all of them.

In [ ]:

def pobs_sar_optimistic_replan(observation_map, gt_hazard_map, people, max_replan_step=15):
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def test_pobs_sar_optimistic1():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['C', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['C', 'U', 'U', 'C', 'U', 'U', 'U'],
        ['C', 'C', 'W', 'U', 'U', 'C', 'C'],
        ['U', 'C', 'W', 'C', 'W', 'U', 'C'],
        ['U', 'U', 'C', 'U', 'U', 'W', 'C'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'C', 'C', 'C', 'C'],
        ['S', 'C', 'W', 'C', 'W', 'C', 'C'],
        ['F', 'S', 'C', 'C', 'C', 'W', 'C'],
        ['S', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (5, 2),
        "p2": (2, 3)
    }
    assert pobs_sar_optimistic_replan(observation_map, gt_hazard_map, people)

test_pobs_sar_optimistic1()

def test_pobs_sar_optimistic2():
    observation_map = np.array([
        ['C', 'U', 'C', 'U', 'U', 'C', 'U'],
        ['C', 'W', 'W', 'U', 'C', 'W', 'W'],
        ['C', 'U', 'U', 'U', 'U', 'U', 'U'],
        ['C', 'C', 'W', 'U', 'U', 'C', 'C'],
        ['U', 'C', 'W', 'U', 'W', 'U', 'C'],
        ['U', 'U', 'C', 'U', 'U', 'W', 'C'],
        ['U', 'W', 'C', 'U', 'W', 'U', 'C']
    ], dtype=np.str0)
    gt_hazard_map = np.array([
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'W', 'W', 'C', 'C', 'W', 'W'],
        ['C', 'C', 'C', 'C', 'C', 'C', 'C'],
        ['C', 'C', 'W', 'S', 'S', 'C', 'C'],
        ['C', 'C', 'W', 'F', 'W', 'C', 'C'],
        ['C', 'C', 'C', 'S', 'C', 'W', 'C'],
        ['C', 'W', 'C', 'C', 'W', 'C', 'C']
    ], dtype=np.str0)
    people = {
        "p1": (5, 2),
        "p2": (2, 3)
    }
    assert pobs_sar_optimistic_replan(observation_map, gt_hazard_map, people)

test_pobs_sar_optimistic2()

print('Tests passed.')